In [6]:
import os
import re
import json
import pandas as pd
# from src.utils import read_config
from prompts import gemini_prompts
import google.generativeai as genai
import google.ai.generativelanguage as glm

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

d:\anaconda\envs\project_envelope\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
organized_result = pd.read_csv('temp_result/organized_result_2.csv')
organized_result.shape

(76, 5)

In [27]:
test_data = organized_result.iloc[:5]
test_data

,document,target,reason,fact,metadata
0,34829.pdf,行政院原住民族委員會及花蓮縣政府,花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實...,據立法委員函轉民眾陳訴行政院原住民族委員會(下稱原民會)補助花蓮區漁會給二予漁民架設衛星導航...,"{'kw': ""['政府', '行政院', '案件', '原住民', '計畫', '漁探機'..."
1,34838.pdf,行政院衛生署,為行政院衛生署辦理九十至九十三年度補、捐助計畫，經該署檢討至少計一二五項計畫應以委辦而竟以補...,本案係立法院鄭委員三元向本院陳訴略以「行政院衛生署將應以委辦執行之經費，改由補助費方式辦理，...,"{'kw': ""['計畫', '捐助', '年度', '行政院', '案件', '意旨', ..."
2,36946.pdf,台北縣萬里鄉公所、台北市政府、基隆市政府、台北縣政府,台北市政府、基隆市政府及台北縣政府未善盡土地使用管制、環境污染稽查及申報資料勾稽、查核之責，...,本案緣陳訴人檢附採證照片陳訴台北市政府核准設立之新生代環境清潔公司（下二稱新生代公司）、統徠...,"{'kw': ""['政府', '公司', '清除', '垃圾', '顯有', '核准', '..."
3,36997.pdf,行政院國軍退除役官兵輔導委員會台北榮民總醫院,行政院國軍退除役官兵輔導委員會台北榮民總醫院辦理「醫學科技大樓法定停車場工程」興建發包作業，...,本案前經審計部函報行政院國軍退除役官兵輔導委員會（以下簡稱退輔會）台北榮民總醫院（以下簡稱台...,"{'kw': ""['工程', '提升', '於九', '法定', '完成', '台北', '..."
4,37008.pdf,行政院國軍退除役官兵輔導委員會,為行政院國軍退除役官兵輔導委員會為改善所屬十一所榮民醫院及十八所安養機構之污水處理廠及衛生下...,行政院國軍退除役官兵輔導委員會﹙以下簡稱退輔會﹚為改善所屬十一所榮民醫院及十八所安養機構之廢...,"{'kw': ""['工程', '政府', '下水道', '方式', '核有', '污水', ..."


In [38]:
input_data = 'reason'
for index, row in test_data.iterrows():
    print(row[input_data])

花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實依政府採購法之規定辦理，採購作業錯誤叢生，補助機關行政院原住民族委員會及花蓮縣政府原住民行政局顯然疏於監督；又花蓮區漁會辦理該採購案，未依規定辦理驗收作業，核銷時亦未檢附初驗紀錄及驗收紀錄，惟花蓮縣政府卻仍予付款，顯未覈實辦理核銷及善盡審核監督之責；該府對於補助案件未訂定明確作業規範，以供受補助單位辦理採購案件時應行遵循事項、採購案件之監辦、經費核銷、及追蹤考核之依據；行政院原住民族委員會對於「原住民新部落運動計畫｜部落社區發展計畫｜東海岸原住民漁業輔導計畫」補助款納入花蓮縣政府預算執行，惟未要求該府訂立補助款作業規範，亦未確實監督其執行情形，而查證工作流於形式；行政院原住民族委員會遲延查復立法委員及本院函詢有關花蓮區漁會辦理衛星定位儀(三合一漁探機)ＧＰＳ採購案，行政效率不彰等情，均有重大違失，爰依監察法第二十四條規定提案糾正
為行政院衛生署辦理九十至九十三年度補、捐助計畫，經該署檢討至少計一二五項計畫應以委辦而竟以補、捐助方式辦理；又其中該署補、捐助受該署監督之財團法人國家衛生研究院至少計二十三項計畫亦有應以委辦而竟以補、捐助方式辦理，核與政府採購法以及預算相關法令規定有違，且該署九十二年之前對補、捐助對象、條件、金額標準、經費用途、審查標準、作業程序、督導考核等未訂定明確、公開及客觀之規範；又該署九十至九十三年度委辦、補、捐助單位，核有連續二年度補助相同單位計有八十案（計畫）、一六三單位；連續三年度補助相同單位計有三十八案(計畫)，一○八單位；連續四年度補助相同單位計有二十四案(計畫)，一三一單位之情事，該署未依行政院所屬各機關委託研究計畫管理辦法第十三條規定，列為該署計畫成效查核重點；另九十年至九十三年該署暨所屬藥物食品檢驗局、中央健康保險局人員等至少十三人次之國外旅費計一百餘萬元，竟於接受該署所補助經費項下列支，其中六人次於返國後復未曾向行政院研考會網站登錄出國報告等，顯係規避「公教人員申請出國案件審核要點」之規定，且與公務員服務法第五條及第二十一條第四款規定意旨亦有二未合等均涉有違失，爰依法提案糾正
台北市政府、基隆市政府及台北縣政府未善盡土地使用管制、環境污染稽查及申報資料勾稽、查核之責，任令轄內違法垃圾轉運站、民營廢棄物清除機構及地方環境保護執行機關長期違規妄

In [20]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [39]:
for index, row in test_data.iterrows():
    
    triple_prompt = PromptTemplate.from_template(gemini_prompts.TRIPLE_PROMPT)
    chain = triple_prompt | llm
    triple_data = {
        "reference": row['reason']
    }

    triple_response = chain.invoke(triple_data)

    triple_response_result = re.sub(r'[```json\n]', '', triple_response.content).replace("實體", "entity").replace("關係", "relationship").split("、")[0]

    to_json_prompt = PromptTemplate.from_template(gemini_prompts.TO_JSON_PROMPT)
    chain = to_json_prompt | llm
    data = {
        "reference": triple_response_result
    }

    to_json_response = chain.invoke(data)
    
    triple_json_result = re.sub(r'[\n]','',to_json_response.content)
    triple_json_result = json.loads(f'[{triple_json_result}]')
    
    test_data['kg'] = triple_json_result
    
    

e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


ValueError: Length of values (3) does not match length of index (5)

In [46]:
test_data['kg'] = str(triple_json_result)

C:\Users\Sean\AppData\Local\Temp\ipykernel_20164\3317704654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['kg'] = str(triple_json_result)


In [47]:
test_data

,document,target,reason,fact,metadata,kg
0,34829.pdf,行政院原住民族委員會及花蓮縣政府,花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實...,據立法委員函轉民眾陳訴行政院原住民族委員會(下稱原民會)補助花蓮區漁會給二予漁民架設衛星導航...,"{'kw': ""['政府', '行政院', '案件', '原住民', '計畫', '漁探機'...","[{'entity1': '花蓮區漁會', 'relationship': '辦理', 'e..."
1,34838.pdf,行政院衛生署,為行政院衛生署辦理九十至九十三年度補、捐助計畫，經該署檢討至少計一二五項計畫應以委辦而竟以補...,本案係立法院鄭委員三元向本院陳訴略以「行政院衛生署將應以委辦執行之經費，改由補助費方式辦理，...,"{'kw': ""['計畫', '捐助', '年度', '行政院', '案件', '意旨', ...","[{'entity1': '花蓮區漁會', 'relationship': '辦理', 'e..."
2,36946.pdf,台北縣萬里鄉公所、台北市政府、基隆市政府、台北縣政府,台北市政府、基隆市政府及台北縣政府未善盡土地使用管制、環境污染稽查及申報資料勾稽、查核之責，...,本案緣陳訴人檢附採證照片陳訴台北市政府核准設立之新生代環境清潔公司（下二稱新生代公司）、統徠...,"{'kw': ""['政府', '公司', '清除', '垃圾', '顯有', '核准', '...","[{'entity1': '花蓮區漁會', 'relationship': '辦理', 'e..."
3,36997.pdf,行政院國軍退除役官兵輔導委員會台北榮民總醫院,行政院國軍退除役官兵輔導委員會台北榮民總醫院辦理「醫學科技大樓法定停車場工程」興建發包作業，...,本案前經審計部函報行政院國軍退除役官兵輔導委員會（以下簡稱退輔會）台北榮民總醫院（以下簡稱台...,"{'kw': ""['工程', '提升', '於九', '法定', '完成', '台北', '...","[{'entity1': '花蓮區漁會', 'relationship': '辦理', 'e..."
4,37008.pdf,行政院國軍退除役官兵輔導委員會,為行政院國軍退除役官兵輔導委員會為改善所屬十一所榮民醫院及十八所安養機構之污水處理廠及衛生下...,行政院國軍退除役官兵輔導委員會﹙以下簡稱退輔會﹚為改善所屬十一所榮民醫院及十八所安養機構之廢...,"{'kw': ""['工程', '政府', '下水道', '方式', '核有', '污水', ...","[{'entity1': '花蓮區漁會', 'relationship': '辦理', 'e..."


In [3]:
configs = read_config('.env/configs.json')
api_key =configs['g_key']
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 500,
    length_function= len
)